In [1]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from cdp_langchain.agent_toolkits import CdpToolkit
from cdp_langchain.utils import CdpAgentkitWrapper


In [2]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

In [3]:
cdp= CdpAgentkitWrapper()
toolkit = CdpToolkit.from_cdp_agentkit_wrapper(cdp)


In [7]:
type(toolkit)

cdp_langchain.agent_toolkits.cdp_toolkit.CdpToolkit

In [30]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(toolkit.get_tools()[1])

CdpTool(name='deploy_contract', description='\nDeploys smart contract with required args: solidity version (string), solidity input json (string), contract name (string), and optional constructor args (Dict[str, Any])\n\nInput json structure:\n{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{}}\n\nYou must set the outputSelection to {"*":{"*":["abi","evm.bytecode"]}} in the settings. The solidity version must be >= 0.8.0 and <= 0.8.28.\n\nSources should contain one or more contracts with the following structure:\n{"contract_name.sol":{"content":"contract code"}}\n\nThe contract code should be escaped. Contracts cannot import from external contracts but can import from one another.\n\nConstructor args are required if the contract has a constructor. They are a key-value\nmap where the key is the arg name and the value is the arg value. Encode uint/int/bytes/string/address values as strings, boolean values as true/false. F

In [42]:
toolkit.get_tools()[1].invoke( input={
    "language":"Solidity",
    "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},
    "sources":{
        "contracts/Verified.sol": {
        "content": "//SPDX-License-Identifier: MIT\r\npragma solidity ^0.8.18;\r\n\r\ncontract Verified {\r\n    string public greet =\r\n        \"Increase this counter if you completed this tutorial\";\r\n    string public tutorial =\r\n        \"https://docs.etherscan.io/contract-verification/multichain-verification\";\r\n\r\n    uint256 public verified = 0;\r\n\r\n    function completedTutorial() public {\r\n        verified += 1;\r\n    }\r\n}\r\n"
      }
    }
}

)

ValidationError: 3 validation errors for DeployContractInput
solidity_version
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
solidity_input_json
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
contract_name
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [23]:
toolkit.get_tools()[1].invoke(input={'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator'})
                              

'Deployed contract Calculator at address 0x7f475E385fbf4ac58734b352f5532bCb509d290a. Transaction link: https://sepolia.basescan.org/tx/0x03cd6cc4732ea81192560bce08fb8b58ed9abf913841fe4aa148d5786fb83a97'

In [60]:
toolkit.get_tools()[1].args_schema.model_json_schema()

{'description': 'Input argument schema for deploy contract action.', 'properties': {'solidity_version': {'description': 'The solidity compiler version', 'title': 'Solidity Version', 'type': 'string'}, 'solidity_input_json': {'description': 'The input json for the solidity compiler', 'title': 'Solidity Input Json', 'type': 'string'}, 'contract_name': {'description': 'The name of the contract class to be deployed', 'title': 'Contract Name', 'type': 'string'}, 'constructor_args': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'The constructor arguments for the contract', 'title': 'Constructor Args'}}, 'required': ['solidity_version', 'solidity_input_json', 'contract_name'], 'title': 'DeployContractInput', 'type': 'object'}

In [4]:
from langgraph.prebuilt import create_react_agent
tools = toolkit.get_tools()
llm = ChatOpenAI(model='gpt-4o-mini')
react_agent = create_react_agent(llm, tools)

In [6]:
from langchain_core.messages import HumanMessage

react_agent.invoke({"messages":[HumanMessage(content="do you have access to any tools to help deploy smart contracts onto the blockchain?")]})

{'messages': [HumanMessage(content='do you have access to any tools to help deploy smart contracts onto the blockchain?', additional_kwargs={}, response_metadata={}, id='cbaab945-aaf6-42e8-9df9-f82e26c68d01'),
  AIMessage(content='Yes, I have the ability to deploy smart contracts onto the blockchain. If you provide me with the details such as the Solidity version, contract code, contract name, and any constructor arguments, I can assist you in deploying the contract.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 4382, 'total_tokens': 4431, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4224}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e6dfccc-5d8e-40ef-846a-22f7d3aca75e-0', usa

In [21]:
react_agent.invoke({"messages":[HumanMessage(content="""please assist deploying a contract named Calculator, using solidity version 0.8.26 using the following contract code: "solidity_input_json":{"language":"Solidity",
                                                            "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},
                                                            "sources":{"contracts/Verified.sol": {
                                                                "content": "// SPDX-License-Identifier: MIT\n\npragma solidity ^0.8.26;\n\ncontract Calculator {\n\n    uint256 result = 0;\n\n    function add(uint256 num) public {\n        result += num;\n    }\n\n    function subtract(uint256 num) public {\n        result -= num;\n    }\n\n    function multiply(uint256 num) public {\n        result *= num;\n    }\n\n    function get() public view returns (uint256) {\n\n        return result;\n    }\n\n}\n"}
                                                            }
                                                            }""")]})

{'messages': [HumanMessage(content='please assist deploying a contract named Calculator, using solidity version 0.8.26 using the following contract code: "solidity_input_json":{"language":"Solidity",\n                                                            "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},\n                                                            "sources":{"contracts/Verified.sol": {\n                                                                "content": "// SPDX-License-Identifier: MIT\n\npragma solidity ^0.8.26;\n\ncontract Calculator {\n\n    uint256 result = 0;\n\n    function add(uint256 num) public {\n        result += num;\n    }\n\n    function subtract(uint256 num) public {\n        result -= num;\n    }\n\n    function multiply(uint256 num) public {\n        result *= num;\n    }\n\n    function get() public view returns (uint256) {\n\n        return result;\n    }\n\n}\n"}\n                                         

In [15]:
toolkit.get_tools()[4].invoke(input={"asset_id":"eth"})

'Balances for wallet 974f8dbf-b045-4058-9d2a-4d10121e1140:\n  0x98750500e358fAa19a78c3f315DfC57eAf32Ce8a: 0.0001'

In [20]:
toolkit.get_tools()[13].invoke(input={"asset_id":"eth",
                                      "wallet":"0x98750500e358fAa19a78c3f315DfC57eAf32Ce8a"})

'Received eth from the faucet. Transaction: https://sepolia.basescan.org/tx/0xc415993522d7782d040be8a850705ce33c02fb4621974bf9f787744112ca9cd6'

In [24]:
contract_template = {'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator'}

In [25]:
contract_template

{'solidity_version': '0.8.26',
 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}',
 'contract_name': 'Calculator'}

In [ ]:
def complete_contract_template(num:int):
    contract_template = {'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":f"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator'}